In [1]:
import os
from fiscalsim_us.api.microsimulation import Microsimulation
from fiscalsim_us.data.datasets import CPS
from fiscalsim_us.tools.dev.taxsim.generate_taxsim_tests import TaxSim35

In [2]:
import numpy as np
import pytest
import pandas as pd
import platform


In [3]:
# Disable warnings
import warnings

warnings.filterwarnings("ignore")

STATES = ["MD", "MA", "NY", "WA"]
DISTANCE = 100
MINIMUM_PERCENT_CLOSE = 0.65


In [5]:
if os.name != "nt":

    @pytest.fixture(scope="module")
    def taxsim():
        taxsim = TaxSim35()

        yield taxsim.generate_from_microsimulation(
            CPS, 2022, None, True, False
        ).set_index("taxsim_taxsimid")

    @pytest.fixture(scope="module")
    def sim():
        yield Microsimulation()

In [6]:
@pytest.mark.skipif(os.name == "nt", reason="This test is not run on Windows")
def test_federal_tax_against_taxsim(sim, taxsim):
    if platform.system() == "Windows":
        warnings.warn("This test is not run on Windows")
        raise pytest.skip()
    tax = sim.calc("income_tax")
    tax.index = sim.calc("tax_unit_id").values
    comparison_df = pd.DataFrame(index=sim.calc("tax_unit_id").values)
    comparison_df["fiscalsim_us"] = tax
    comparison_df["taxsim"] = taxsim.taxsim_fiitax
    relative_distance = np.absolute(
        comparison_df.fiscalsim_us - comparison_df.taxsim
    )
    percent_close = (relative_distance < DISTANCE).mean()
    assert percent_close > MINIMUM_PERCENT_CLOSE

In [7]:
@pytest.mark.skipif(os.name == "nt", reason="This test is not run on Windows")
@pytest.mark.parametrize("state", STATES)
def test_state_income_tax_against_taxsim(state: str, sim, taxsim):
    in_state = sim.calc("tax_unit_state").values == state
    tax = sim.calc("state_income_tax")
    tax.index = sim.calc("tax_unit_id").values
    comparison_df = pd.DataFrame(
        dict(
            fiscalsim_us=tax,
            taxsim=taxsim.taxsim_siitax,
        ),
        index=sim.calc("tax_unit_id").values,
    )
    comparison_df = comparison_df[in_state]
    relative_distance = np.absolute(
        comparison_df.fiscalsim_us - comparison_df.taxsim
    )
    percent_close = (relative_distance < DISTANCE).mean()
    assert percent_close > MINIMUM_PERCENT_CLOSE
